In [1]:
!pip install tqdm

In [2]:
import os
import requests
from tqdm import tqdm # Barre de progression (à installer via !pip install tqdm)

# Création du dossier de stockage BRUT (Bronze)
output_dir = "nyc_taxi_bronze"
os.makedirs(output_dir, exist_ok=True)

# L'année qu'on veut récupérer
year = 2023
months = range(1, 13) # De 1 à 12

print(f"--- Démarrage du téléchargement pour {year} ---")

for month in months:
    # Format du mois sur 2 chiffres (ex: 01, 02...)
    month_str = f"{month:02d}"
    filename = f"yellow_tripdata_{year}-{month_str}.parquet"
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{filename}"
    filepath = os.path.join(output_dir, filename)
    
    # On ne télécharge pas si le fichier existe déjà
    if not os.path.exists(filepath):
        print(f"Téléchargement de {filename}...")
        response = requests.get(url, stream=True)
        
        # Écriture du fichier par blocs
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024*1024): # Blocs de 1Mo
                if chunk:
                    f.write(chunk)
    else:
        print(f"{filename} déjà présent.")

print("--- Téléchargement terminé ! ---")

--- Démarrage du téléchargement pour 2023 ---
Téléchargement de yellow_tripdata_2023-01.parquet...
Téléchargement de yellow_tripdata_2023-02.parquet...
Téléchargement de yellow_tripdata_2023-03.parquet...
Téléchargement de yellow_tripdata_2023-04.parquet...
Téléchargement de yellow_tripdata_2023-05.parquet...
Téléchargement de yellow_tripdata_2023-06.parquet...
Téléchargement de yellow_tripdata_2023-07.parquet...
Téléchargement de yellow_tripdata_2023-08.parquet...
Téléchargement de yellow_tripdata_2023-09.parquet...
Téléchargement de yellow_tripdata_2023-10.parquet...
Téléchargement de yellow_tripdata_2023-11.parquet...
Téléchargement de yellow_tripdata_2023-12.parquet...
--- Téléchargement terminé ! ---


# Nettoyage et Consolidation (Bronze vers Argent

Une fois que vous avez 12 fichiers (un par mois), le but est de les fusionner en un seul dataset géant, optimisé pour votre Mac Pro.

Défis techniques à résoudre avec Spark :

Valeurs aberrantes : Supprimer les courses avec trip_distance <= 0 ou total_amount < 0.

Partitionnement : Fusionner les 12 mois et re-sauvegarder le tout partitionné par Jour ou par Semaine.

Typage : S'assurer que les dates sont bien au format Timestamp.